Problem Statement.

Design a data structure to store the strings' count with the ability to return the strings with minimum and maximum counts.

Implement the AllOne class:

    AllOne() Initializes the object of the data structure.
    inc(String key) Increments the count of the string key by 1. If key does not exist in the data structure, insert it with count 1.
    dec(String key) Decrements the count of the string key by 1. If the count of key is 0 after the decrement, remove it from the data structure. It is guaranteed that key exists in the data structure before the decrement.
    getMaxKey() Returns one of the keys with the maximal count. If no element exists, return an empty string "".
    getMinKey() Returns one of the keys with the minimum count. If no element exists, return an empty string "".

 

Example 1:

Input
["AllOne", "inc", "inc", "getMaxKey", "getMinKey", "inc", "getMaxKey", "getMinKey"]
[[], ["hello"], ["hello"], [], [], ["leet"], [], []]
Output
[null, null, null, "hello", "hello", null, "hello", "leet"]

Explanation
AllOne allOne = new AllOne();
allOne.inc("hello");
allOne.inc("hello");
allOne.getMaxKey(); // return "hello"
allOne.getMinKey(); // return "hello"
allOne.inc("leet");
allOne.getMaxKey(); // return "hello"
allOne.getMinKey(); // return "leet"

 

Constraints:

    1 <= key.length <= 10
    key consists of lowercase English letters.
    It is guaranteed that for each call to dec, key is existing in the data structure.
    At most 3 * 10^4 calls will be made to inc, dec, getMaxKey, and getMinKey.

 

Follow up: Could you apply all the operations in O(1) time complexity?

# Two HashTables - O(N) dec, others O(1) runtime, O(N) space

In [2]:
from collections import defaultdict

class AllOne:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.keyDict = defaultdict(int)
        self.freqDict = defaultdict(set)
        self.maxFreq = 0
        self.minFreq = 0  

    def inc(self, key: str) -> None:
        """
        Inserts a new key <Key> with value 1. Or increments an existing key by 1.
        """
        prevFreq = self.keyDict.get(key, 0)
        newFreq = prevFreq + 1
        self.keyDict[key] = newFreq
        if prevFreq > 0 and prevFreq in self.freqDict:
            self.freqDict[prevFreq].remove(key)
            if len(self.freqDict[prevFreq]) == 0: self.freqDict.pop(prevFreq)
        self.freqDict[newFreq].add(key)
        
        if prevFreq == self.maxFreq: self.maxFreq += 1
        if self.minFreq == 0 or self.minFreq not in self.freqDict: self.minFreq += 1
        if newFreq < self.minFreq: self.minFreq = newFreq

    def dec(self, key: str) -> None:
        """
        Decrements an existing key by 1. If Key's value is 1, remove it from the data structure.
        """
        prevFreq = self.keyDict.get(key, 0)
        newFreq = prevFreq - 1
        self.keyDict[key] = newFreq
        if self.keyDict[key] == 0: self.keyDict.pop(key)
            
        self.freqDict[prevFreq].remove(key)
        if len(self.freqDict[prevFreq]) == 0: self.freqDict.pop(prevFreq)
        if newFreq > 0:
            self.freqDict[newFreq].add(key)
        
        if self.maxFreq not in self.freqDict: self.maxFreq -= 1
        if prevFreq == self.minFreq and self.minFreq not in self.freqDict: 
            self.minFreq = min(self.freqDict.keys())
        

    def getMaxKey(self) -> str:
        """
        Returns one of the keys with maximal value.
        """
        if self.maxFreq == 0: return ""
        for val in self.freqDict[self.maxFreq]: return val
        

    def getMinKey(self) -> str:
        """
        Returns one of the keys with Minimal value.
        """
        if self.minFreq == 0: return ""
        for val in self.freqDict[self.minFreq]: return val

# Doubly Linked List and Hash Table - O(1) runtime, O(N) space

In [4]:
from collections import defaultdict

class Node(object):
    def __init__(self, val=0):
        self.num = val
        self.key_set = set()
        self.prev = None
        self.next = None
        
class AllOne(object):
    def __init__(self):
        self.head = Node() 
        self.tail = Node()  
        self.head.next = self.tail
        self.tail.prev = self.head
        self.cache = defaultdict(Node)  

    def inc(self, key):
        """
        Inserts a new key <Key> with value 1. Or increments an existing key by 1.
        """
        if not key in self.cache:  
            cur = self.head
        else:
            cur = self.cache[key]
            cur.key_set.remove(key)

        if cur.num + 1 != cur.next.num:  
            new_node = Node(cur.num + 1)
            self._insert_after(cur, new_node)
        else:
            new_node = cur.next

        new_node.key_set.add(key)  
        self.cache[key] = new_node  

        if not cur.key_set and cur.num != 0:  
            self._remove(cur)

    def dec(self, key):
        """
        Decrements an existing key by 1. If Key's value is 1, remove it from the data structure.
        """
        if not key in self.cache:
            return

        cur = self.cache[key]
        self.cache.pop(key)  
        cur.key_set.remove(key)

        if cur.num != 1:
            if cur.num - 1 != cur.prev.num:  
                new_node = Node(cur.num - 1)
                self._insert_after(cur.prev, new_node)
            else:
                new_node = cur.prev
            new_node.key_set.add(key)
            self.cache[key] = new_node

        if not cur.key_set and cur.num != 0:  
            self._remove(cur)

    def getMaxKey(self):
        """
        Returns one of the keys with maximal value.
        """
        if self.tail.prev.num == 0:
            return ""
        key = self.tail.prev.key_set.pop()  # pop and add back to get arbitrary (but not random) element
        self.tail.prev.key_set.add(key)
        return key

    def getMinKey(self):
        """
        Returns one of the keys with Minimal value.
        """
        if self.head.next.num == 0:
            return ""
        key = self.head.next.key_set.pop()
        self.head.next.key_set.add(key)
        return key

    def _remove(self, node):
        p = node.prev
        n = node.next
        p.next = n
        n.prev = p

    def _insert_after(self, node, new_block):
        old_after = node.next
        node.next = new_block
        new_block.prev = node
        new_block.next = old_after
        old_after.prev = new_block